# Kaggle Submission

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import pickle
import csv
%matplotlib inline

import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.linear_model import Ridge

np.random.seed(42)


## Load in Trained Model and Scaler

In [2]:
# use pickle.load to load in these assets

with open('../assets/ridge.pkl', 'rb') as f:
    ridge = pickle.load(f)
with open('../assets/scaler.pkl', 'rb') as f:
    ss = pickle.load(f)

In [3]:
ridge

RidgeCV(alphas=array([ 0.1,  0.2, ...,  9.9, 10. ]), cv=None,
    fit_intercept=True, gcv_mode=None, normalize=False, scoring=None,
    store_cv_values=False)

## Load Test Data as df

In [81]:
# read in our test.csv as `kaggle`
df = pd.read_csv('../datasets/test.csv', index_col='Id')

In [ ]:
df.shape

## Clean data in same method as training data

In [5]:
# we excluded objects, filled nulls with 0, and dropped `PID`
with open('../assets/columns.pkl', 'rb') as f:
    included = pickle.load(f)

In [6]:
df.shape

(879, 79)

### Lowercase Columns and Replace Spaces with Underscores

In [50]:
# df.columns = included

In [12]:
df.columns = map(str.lower, df.columns)

In [13]:
df.columns = df.columns.str.replace(' ', '_')

In [72]:
df.rename(columns = {'saleprice':'SalePrice'}, inplace=True)

In [65]:
df.columns

Index(['ms_subclass', 'ms_zoning', 'lot_frontage', 'lot_area', 'street',
       'alley', 'lot_shape', 'land_contour', 'utilities', 'lot_config',
       'land_slope', 'neighborhood', 'condition_1', 'condition_2', 'bldg_type',
       'house_style', 'overall_qual', 'overall_cond', 'year_built',
       'year_remod/add', 'roof_style', 'roof_matl', 'exterior_1st',
       'exterior_2nd', 'mas_vnr_type', 'mas_vnr_area', 'exter_qual',
       'exter_cond', 'foundation', 'bsmt_qual', 'bsmt_cond', 'bsmt_exposure',
       'bsmtfin_type_1', 'bsmtfin_sf_1', 'bsmtfin_type_2', 'bsmtfin_sf_2',
       'bsmt_unf_sf', 'total_bsmt_sf', 'heating', 'heating_qc', 'central_air',
       'electrical', '1st_flr_sf', '2nd_flr_sf', 'low_qual_fin_sf',
       'gr_liv_area', 'bsmt_full_bath', 'bsmt_half_bath', 'full_bath',
       'half_bath', 'bedroom_abvgr', 'kitchen_abvgr', 'kitchen_qual',
       'totrms_abvgrd', 'functional', 'fireplaces', 'fireplace_qu',
       'garage_type', 'garage_finish', 'garage_cars', 'garage

## Create a Dictionary for Null Values and Empty Values

In [16]:
na_dict = {'alley':'None',
'fireplace_qu':'None',
'garage_cond':'None',
'pool_qc':'None',
'fence':'None',
'misc_feature':'None',
'lot_frontage':lambda x: .01*x['lot_area'],
'mas_vnr_type':'None',
'mas_vnr_area':0,
'bsmt_qual':'None',
'bsmt_cond':'None',
'bsmt_exposure':'None',
'bsmtfin_type_1':'None',
'bsmtfin_sf_1':0,
'bsmtfin_type_2':'None',
'bsmtfin_sf_2':0,
'bsmt_unf_sf':0,
'total_bsmt_sf':0,
'bsmt_full_bath':'None',
'bsmt_half_bath':'None',
'garage_type':'None',
'garage_finish':'None',
'garage_qual':'None',
'garage_cars':0,
'garage_area':0}

In [25]:
df.drop(['garage_yr_blt'], axis=1, inplace=True)

## Drop PID & Exclude Objects

In [26]:
df.drop(['pid'], axis=1, inplace=True)

In [ ]:
df = df.select_dtypes(exclude='object')

In [27]:
df_dummies = pd.get_dummies(df)

In [80]:
# set(included).difference(df.columns)  #Code to run

In [29]:
set(included).difference(df_dummies.columns)  #Code to run

{'bsmt_full_bath_0.0',
 'bsmt_full_bath_1.0',
 'bsmt_half_bath_0.0',
 'bsmt_half_bath_1.0',
 'overall_cond_5',
 'overall_cond_6',
 'overall_cond_7',
 'overall_qual_4',
 'overall_qual_5',
 'overall_qual_6',
 'overall_qual_7',
 'overall_qual_8'}

In [52]:
df_dummies.columns.value_counts()

functional_Min2         1
totrms_abvgrd           1
exterior_1st_AsbShng    1
pool_qc_Ex              1
lot_frontage            1
bedroom_abvgr           1
overall_qual_5          1
roof_matl_WdShngl       1
electrical_FuseA        1
exterior_2nd_AsbShng    1
exterior_2nd_CBlock     1
overall_qual            1
roof_style_Shed         1
roof_matl_Roll          1
pool_qc_None            1
garage_qual_Fa          1
neighborhood_MeadowV    1
bsmt_exposure_Mn        1
lot_area                1
bsmtfin_type_1_Rec      1
neighborhood_Edwards    1
sale_type_ConLw         1
street_Grvl             1
exterior_1st_Plywood    1
bsmt_full_bath          1
bsmt_full_bath_1.0      1
roof_style_Hip          1
neighborhood_SawyerW    1
neighborhood_Crawfor    1
bsmt_qual_Fa            1
                       ..
garage_qual_Gd          1
functional_Typ          1
sale_type_ConLD         1
kitchen_qual_Fa         1
overall_cond_6          1
garage_qual_Po          1
exter_qual_Gd           1
roof_style_M

## Assign values to missing columns df_dummies

In [30]:
df_dummies['bsmt_full_bath_0.0']=0
df_dummies['bsmt_full_bath_1.0']=0
df_dummies['bsmt_half_bath_0.0']=0
df_dummies['bsmt_half_bath_1.0']=0
df_dummies['overall_cond_5']=0 
df_dummies['overall_cond_6']=0
df_dummies['overall_cond_7']=0
df_dummies['overall_qual_4']=0
df_dummies['overall_qual_5']=0
df_dummies['overall_qual_6']=0
df_dummies['overall_qual_7']=0
df_dummies['overall_qual_8']=0

In [55]:
(ss.mean_).shape

(134,)

In [74]:
# use our previously fit scaler to transform our kaggle data, save as `kaggle_sc`
df_sc = ss.transform(df_dummies[included])

## Make Predictions

In [75]:
preds = ridge.predict(df_sc)

## Align predictions back with index and set up header

In [76]:
submission = pd.DataFrame(preds, index=df_dummies[included].index, columns=['SalePrice'])

## Sort index (required for proper submission)

In [77]:
submission.sort_index(inplace=True)

## Save to csv to submit

In [78]:
# use pandas .to_csv to save these predictions out as `../data/numeric_only_lr.csv`
submission.to_csv('../datasets/dummies_ridge.csv')

## Use command line `head` to check data is correct format

In [79]:
!head ../datasets/dummies_ridge.csv

Id,SalePrice
2,123245.74672502995
4,286954.9299571526
6,215626.9516779511
7,202405.77793382164
17,191871.05603488185
18,336811.6383799143
22,211094.024389459
27,144982.26718989923
31,123662.09559337833
